In [3]:
%matplotlib inline

import json
import pandas as pd
import matplotlib.pyplot as plt

## Load the Twitter Trend Snapshots

Take the raw fata from dynamoDB dump, clean it up, and load it into a pandas data frame.

In [43]:
with open('output.json', 'rb') as f:
    trends = json.load(f)

In [7]:
# use this to inform cleanup process
trends["Items"][0]

{u'placeID': {u'N': u'2407517'},
 u'trend': {u'M': {u'date': {u'S': u'2016-03-03'},
   u'mappedName': {u'S': u'world book day'},
   u'placeName': {u'S': u'Fresno'},
   u'tweetName': {u'S': u'#WorldBookDay'},
   u'tweetVolume': {u'N': u'225775'},
   u'url': {u'S': u'http://twitter.com/search?q=%23WorldBookDay'}}},
 u'trendTime': {u'N': u'1457022783'}}

In [44]:
trends_clean = []

In [45]:
for trend in trends["Items"]:
    new = {}
    new["placeID"] = trend["placeID"]["N"]
    new["trendMapped"] = trend["trend"]["M"]["mappedName"]["S"]
    new["placeName"] = trend["trend"]["M"]["placeName"]["S"]
    new["trendName"] = trend["trend"]["M"]["tweetName"]["S"]
    new["trendVolume"] = trend["trend"]["M"]["tweetVolume"]["N"]
    new["searchUrl"] = trend["trend"]["M"]["url"]["S"]
    new["trendDate"] = trend["trend"]["M"]["date"]["S"]
    new["trendTime"] = trend["trendTime"]["N"]
    trends_clean.append(new)

In [46]:
# it worked
trends_clean[0]

{'placeID': u'2407517',
 'placeName': u'Fresno',
 'searchUrl': u'http://twitter.com/search?q=%23WorldBookDay',
 'trendDate': u'2016-03-03',
 'trendMapped': u'world book day',
 'trendName': u'#WorldBookDay',
 'trendTime': u'1457022783',
 'trendVolume': u'225775'}

In [47]:
df_trends = pd.DataFrame(trends_clean)

## Pandas Cleanup
Converting columns and shit

In [48]:
df_trends.trendTime = df_trends.trendTime.astype(int)
df_trends.trendVolume = df_trends.trendVolume.astype(int)
df_trends.index = pd.to_datetime(df_trends.trendTime, unit = 's')
df_trends = df_trends.drop("trendTime", axis = 1)

In [49]:
df_trends[(df_trends["placeName"] == "New York") & (df_trends["trendDate"] == '2016-03-10')].sort_values(by = "trendVolume", ascending = False)

,placeID,placeName,searchUrl,trendDate,trendMapped,trendName,trendVolume
trendTime,,,,,,,
2016-03-10 11:51:15,2459115,New York,http://twitter.com/search?q=%23DemDebate,2016-03-10,dem debate,#DemDebate,536268
2016-03-10 11:20:55,2459115,New York,http://twitter.com/search?q=%23DemDebate,2016-03-10,dem debate,#DemDebate,533930
2016-03-10 11:03:58,2459115,New York,http://twitter.com/search?q=%23DemDebate,2016-03-10,dem debate,#DemDebate,532758
2016-03-10 10:33:01,2459115,New York,http://twitter.com/search?q=%23DemDebate,2016-03-10,dem debate,#DemDebate,530332
2016-03-10 10:21:17,2459115,New York,http://twitter.com/search?q=%23DemDebate,2016-03-10,dem debate,#DemDebate,530020
2016-03-10 07:03:01,2459115,New York,http://twitter.com/search?q=%23DemDebate,2016-03-10,dem debate,#DemDebate,506155
2016-03-10 03:48:02,2459115,New York,http://twitter.com/search?q=%23DemDebate,2016-03-10,dem debate,#DemDebate,318096
2016-03-10 20:33:02,2459115,New York,http://twitter.com/search?q=%22Like+I+Would%22,2016-03-10,Like I Would,Like I Would,255270
2016-03-10 22:04:06,2459115,New York,http://twitter.com/search?q=Marvel,2016-03-10,marvel,Marvel,164611


In [52]:
df_trends.to_json("cleaned_trends.json",orient = 'records')


'[{"placeID":"2407517","placeName":"Fresno","searchUrl":"http:\\/\\/twitter.com\\/search?q=%23WorldBookDay","trendDate":"2016-03-03","trendMapped":"world book day","trendName":"#WorldBookDay","trendVolume":225775},{"placeID":"2407517","placeName":"Fresno","searchUrl":"http:\\/\\/twitter.com\\/search?q=%23WorldBookDay","trendDate":"2016-03-03","trendMapped":"world book day","trendName":"#WorldBookDay","trendVolume":271430},{"placeID":"2407517","placeName":"Fresno","searchUrl":"http:\\/\\/twitter.com\\/search?q=Romney","trendDate":"2016-03-03","trendMapped":"romney","trendName":"Romney","trendVolume":525041},{"placeID":"2407517","placeName":"Fresno","searchUrl":"http:\\/\\/twitter.com\\/search?q=%23WorldBookDay","trendDate":"2016-03-03","trendMapped":"world book day","trendName":"#WorldBookDay","trendVolume":328667},{"placeID":"2407517","placeName":"Fresno","searchUrl":"http:\\/\\/twitter.com\\/search?q=%23GIFparty","trendDate":"2016-03-04","trendMapped":"gi fparty","trendName":"#GIFpart